<hr>

**Authors:**  Thomas Duthoit<br>
**Copyright:** 2022 Thomas Duthoit <br>
**License:** MIT

<hr>

<center><h1>Anomaly detection in satellite image time series related to forest monitoring<\h1><\center>

<hr>

## Introduction

## Data used in this notebook

| Product Description | WEkEO HDA ID | WEkEO metadata |
|:--------------------:|:-----------------:|:-----------------:|
| Sentinel-2 MSI level-1C | EO:EUM:DAT:0411 | <a href="https://navigator.eumetsat.int/product/EO:EUM:DAT:SENTINEL-3:SL_1_RBT___NTC?query=SLSTR&s=advanced" target="_blank">link</a> | EO:ESA:DAT:SENTINEL-2:MSI | <a href="https://www.wekeo.eu/data?view=dataset&dataset=EO%3AESA%3ADAT%3ASENTINEL-2%3AMSI" target="_blank">link</a> |

## Learning outcomes

At the end of this notebook you will know;
* How a certain type of data is collected by satellites.
* How you can use a certain type of data for a specific environmental application.
* How to visualise a certain type of data.
* How to analyse a certain type of data with a certain type of method.

## Further resources

## Notebook outline
    
 1. [Load Sentinel-2 level 1C tiles on WEkEO's catalogue](#section1)
 2. [Generate the time series : compute NDVI and cloud mask](#section2)
 3. [Superpixel segmentation of the area of interest](#section3)
 4. [Compute the NDVI mean and cloud mask mean for each superpixel](#section4)
 5. [Filtering : keep only forest superpixels](#section5)
 6. [Apply BFAST algorithm to all remaining time series](#section6)
 7. [USE CASE 1 : Forest fire detection in Var region (France)](#section7)
 8. [USE CASE 2 : Monitor parasite attacks on trees, example in Lot region (France)](#section8)


<hr>

In [6]:
# library imports (some common examples used and available on the WEkEO Jupyter Lab)
import cartopy.crs as ccrs
import glob
import inspect
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np
import os
import warnings
import xarray as xr



## <a id='section1'></a>1. Load Sentinel-2 level 1C tiles on WEkEO's catalogue
[Back to top](#TOC_TOP)


The first step is to download the data from the WEkEO catalogue : <a href="https://www.wekeo.eu/data?view=catalogue&initial=1" target="_blank">WEkEO Catalogue</a>. To do so, we can run a small python script which summarizes the all the options :
- time period
- tile name : (Var region, for use cas 1)

If you need any help with the Harmonized Data Access (HDA), WEkEO provides a well guided documentation  <a href="https://www.wekeo.eu/docs/harmonised-data-access-api" target="_blank">here</a>.

Once all the data is downloaded, we need to unzip the file that will be usefull to our use cases :

- band 4 :
- band 8 :
- cloud mask : 

The result of both processes gives us everything we need to start our process. The data is stored in the S2_data folder. The next step is to store the names of all these files into a list 

**Note:** You can go through the S2_data folder and ...

In [8]:
spatial_files = []
directory = 'S2_data/Lot/'
for path, subdirs, files in os.walk(directory):
    for name in files:
        if name[-5:] == "0.gml" or name[-3:] == "jp2":
            file_path = os.path.join(path, name)
            spatial_files.append(file_path)
print('number of files = ', len(spatial_files))
print('number of acquisitions = ', int(len(spatial_files)/3))

number of files =  0
number of acquisitions =  0


We have now a list _`spatial_files`_ containing all file paths

## <a id='section2'></a>2. Generate the time series : compute NDVI and cloud mask
[Back to top](#TOC_TOP)

The NDVI is ...

<center>$NDVI = \frac{NIR - RED}{NIR + RED}$</center>
    

The following piece of code will run through every single file and compute the NDVI and the cloud mask for every acquisition. Here is an explanation of the important output of the cell :
    
- ndvi_stack
- clc_stack
- bbox
- timestamp

In [ ]:
bbox_32 = bbox.transform(CRS.UTM_31N)

timestamp = []
for i in range(0, len(spatial_files), 3):
    print("Processing file =", i, ',', i+1, ' &', i+2)
    year = spatial_files[i][4:8]
    month = spatial_files[i][9:11]
    day = spatial_files[i][11:13]

    jp1 = rasterio.open(zipfiles[i])
    jp2 = rasterio.open(zipfiles[i+1])
    
    jp1_clipped_r, jp1_transform = rasterio.mask.mask(jp1, [bbox_32.geometry], crop=True)
    out_meta = jp1.meta
    jp2_clipped_r, jp2_transform = rasterio.mask.mask(jp2, [bbox_32.geometry], crop=True)
    
    jp1_clipped = jp1_clipped_r.squeeze()
    jp2_clipped = jp2_clipped_r.squeeze()
    jp1_clipped = jp1_clipped.astype('int16')
    jp2_clipped = jp2_clipped.astype('int16')

    ndvi = ((jp2_clipped - jp1_clipped) / (jp2_clipped + jp1_clipped)).astype('float32')
    try:
        clm = gpd.read_file(spatial_files[i+2])

        clm_clipped = clm.intersection(bbox_32.geometry)
        clm_clipped = clm_clipped[~clm_clipped.is_empty]
        
        if len(clm_clipped.index) == 0:
            clip_clm = np.zeros(jp1_clipped.shape)
        
        else:
            
            out_meta.update({"height": jp1_clipped_r.shape[1],
                 "width": jp1_clipped_r.shape[2],
                 "transform": out_transform})
            with rasterio.open("file.jp2", "w", **out_meta) as dest:
                dest.write(jp1_clipped_r)
            g = rasterio.open("file.jp2")
            clip_clm, _ = rasterio.mask.mask(g, clm_clipped)

        clc_mask = np.where(clip_clm <= 0, 0, 1).astype('uint8')

    except ValueError:
        clip_clm = np.zeros(jp1_clipped.shape)
        clc_mask = np.where(clip_clm <= 0, 0, 1).astype('uint8')
    
    if i==0:
        ndvi_stack = np.expand_dims(ndvi, axis=0)
        clc_mask_stack = np.expand_dims(clc_mask.squeeze(), axis=0)
    else:
        ndvi_stack = np.vstack((ndvi_stack, np.expand_dims(ndvi, axis=0)))
        clc_mask_stack = np.vstack((clc_mask_stack, np.expand_dims(clc_mask.squeeze(), axis=0)))

    timestamp.append(datetime(int(year), int(month), int(day)))

In [ ]:
print(ndvi_stack.shape)
print(clc_mask.shape)

Explain the dimension of both variables

In [ ]:
eopatch = EOPatch()

eopatch[FeatureType.DATA, "NDVI"] = np.expand_dims(ndvi_stack, axis=3)
eopatch[FeatureType.MASK, "CLM"] = np.expand_dims(clc_mask_stack, axis=3)
eopatch[FeatureType.BBOX] = BBox(bbox_32.geometry.bounds, CRS.UTM_31N)
eopatch[FeatureType.TIMESTAMP] = timestamp

eopatch.save('Patches/Var/Raw_data')

In [ ]:
eopatch

## <a id='section3'></a>3. Superpixel segmentation of the area of interest
[Back to top](#TOC_TOP)


Segmenting in superpixels has two major benefits. The first one is that we considerably reduce the dimension and so the number of time series we will have to process. The second reason is that, from an analytic point of view, it has no real benefit to proceed pixel by pixel.

In [ ]:
slic_segmentation_task = SlicSegmentationTask((FeatureType.DATA, 'NDVI_STANDARDIZED'), 
                                       (FeatureType.MASK_TIMELESS, 'SUPER_PIXELS'),
                                      n_segments=10000, compactness=0.0001, sigma=1)

raster_to_vector_task = RasterToVectorTask(features=(FeatureType.MASK_TIMELESS, 'SUPER_PIXELS', 'SUPER_PIXELS'))

In [2]:
# standardize the NDVI
eopatch.data['NDVI_STANDARDIZED'] = stats.zscore(eopatch.data['NDVI'], nan_policy='omit')

# freeing memory
del eopatch.data['NDVI']

# execute the superpixel task
eopatch = slic_segmentation_task.execute(eopatch)
eopatch.mask_timeless['SUPER_PIXELS'] = eopatch.mask_timeless['SUPER_PIXELS'].astype(np.int32)
# execute the raster to vector task
eopatch = raster_to_vector_task.execute(eopatch)

# save the processed eopatch
eopatch.save("/home/thomas/work/Etude CNES/WEkEO data/Communes Lot processed/Bellefont-La Rauze")

In [ ]:
# print result of the segmentation

Explanation

## <a id='section4'></a>4. Compute the NDVI mean and cloud mask mean for each superpixel
[Back to top](#TOC_TOP)

In this section, we are going to construct our time series. Each superpixel will have his own time serie from January 2018 to December 2021. To do so, we need to compute the NDVI mean of every pixel contained in each superpixel. We proceed the same way for the cloud mask. 

However, it is possible that for a specific date, there is too much cloud coverage on a superpixel, meaning that the NDVI value computed is not relevant. 

What we do is, if the mean of the cloud coverage of one superpixel at a specific date is superior to 0.2, then we set to NaN the NDVI mean of the corresponding superpixel and date.valid_data, dates = get_valid_data(eopatch=eopatch)

In [ ]:
valid_data, dates = get_valid_data(eopatch=eopatch)

In [ ]:
valid_data.shape

We have now constructed our NDVI time series. valid_data is shape (..). (..) is the number of Sentinel-2 acquisition (e.g. dates) and (..) is the number of superpixels. Each superpixel has his own NDVI time series.

## <a id='section5'></a>5. Filtering : keep only forest superpixels
[Back to top](#TOC_TOP)

The area of interest (..) is not full forest area. This means that among our time series, there are some relevant to analyse in our case. In this section, you can apply a filter in order to only keep the forest time series.

- shapefile : file where is stored the polygons that delimit the forest area in the Var departement (France).
- filter_percentage : percentage to which we keep a time series. If one superpixel is covered of this percentage or more of forest area, it is kept. Otherwise, it is dropped. Default is 0.8.


In [ ]:
time_series_forest, ts_indices_preserved = get_forest_time_series(eopatch=eopatch, shapefile='../../work/Etude CNES/Forêts Lot/FORMATION_VEGETALE_D046.shp', filter_percentage=0.8)

In [ ]:
# print forest shapefile of the aoi and next to it the superpixels kept

## <a id='section6'></a>6. Apply BFAST algorithm to all remaining time series
[Back to top](#TOC_TOP)

Little paragraph on how BFAST works

+ links to documentation

In [ ]:
end_train = datetime(2020,1,1)
start_monitor = datetime(2021,1,1)
end_monitor = datetime(2021,6,30)
bfast_model, valid_data_f, dates_f = set_bfast_params(valid_data, dates, ts_indices_preserved, end_train, start_monitor, end_monitor)

In [ ]:
valid_data_f.shape

In [ ]:
#%%capture
breaks, magnitudes, means, valids = execute_bfast(bfast_model, valid_data_f, dates_f)

In [11]:
print("number of breaks detected :", (breaks>0).sum())
print("percentage of time series detected with an anomaly :", ((breaks>0).sum()/len(breaks))*100)

NameError: name 'breaks' is not defined

## <a id='section7'></a>7. USE CASE 1 : Forest fire detection in Var region (France)
[Back to top](#TOC_TOP)

In [ ]:
results = organise_results(time_series_forest, dates_f, start_monitor, breaks, magnitudes)
results.head()

In [ ]:
breakpoint_df = group_by_breakpoints(results)
breakpoint_df.head()

## <a id='section8'></a>8. USE CASE 2 : Monitor parasite attacks on trees, example in Lot region (France)
[Back to top](#TOC_TOP)

<div class="alert alert-block alert-warning">

### Challenge:

A final section of your notebook could invite the user to try and adapt your notebook for their own purposes. This is not necessary but can really help people to learn by applying their new knowled
 <div>

In [10]:
# Enter your solution here


<hr>